In [1]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset,DataLoader
import time
%matplotlib inline

In [2]:
def imshow(inp,cmap=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)

In [3]:
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

In [6]:
train = ImageFolder("/mnt/c/Users/hyoja/OneDrive/문서/kaggle_dataset/animals151/dataset/dataset", simple_transform)
valid = ImageFolder("/mnt/c/Users/hyoja/OneDrive/문서/kaggle_dataset/animals151/dataset/dataset", simple_transform)

In [8]:
print(train.class_to_idx)

{'acinonyx-jubatus': 0, 'aethia-cristatella': 1, 'agalychnis-callidryas': 2, 'agkistrodon-contortrix': 3, 'ailuropoda-melanoleuca': 4, 'ailurus-fulgens': 5, 'alces-alces': 6, 'anas-platyrhynchos': 7, 'ankylosaurus-magniventris': 8, 'apis-mellifera': 9, 'aptenodytes-forsteri': 10, 'aquila-chrysaetos': 11, 'ara-macao': 12, 'architeuthis-dux': 13, 'ardea-herodias': 14, 'balaenoptera-musculus': 15, 'betta-splendens': 16, 'bison-bison': 17, 'bos-gaurus': 18, 'bos-taurus': 19, 'bradypus-variegatus': 20, 'branta-canadensis': 21, 'canis-lupus': 22, 'canis-lupus-familiaris': 23, 'carcharodon-carcharias': 24, 'cardinalis-cardinalis': 25, 'cathartes-aura': 26, 'centrochelys-sulcata': 27, 'centruroides-vittatus': 28, 'ceratitis-capitata': 29, 'ceratotherium-simum': 30, 'chelonia-mydas': 31, 'chrysemys-picta': 32, 'circus-hudsonius': 33, 'codium-fragile': 34, 'coelacanthiformes': 35, 'colaptes-auratus': 36, 'connochaetes-gnou': 37, 'correlophus-ciliatus': 38, 'crocodylus-niloticus': 39, 'crotalus-a

In [9]:
print(train.classes)

['acinonyx-jubatus', 'aethia-cristatella', 'agalychnis-callidryas', 'agkistrodon-contortrix', 'ailuropoda-melanoleuca', 'ailurus-fulgens', 'alces-alces', 'anas-platyrhynchos', 'ankylosaurus-magniventris', 'apis-mellifera', 'aptenodytes-forsteri', 'aquila-chrysaetos', 'ara-macao', 'architeuthis-dux', 'ardea-herodias', 'balaenoptera-musculus', 'betta-splendens', 'bison-bison', 'bos-gaurus', 'bos-taurus', 'bradypus-variegatus', 'branta-canadensis', 'canis-lupus', 'canis-lupus-familiaris', 'carcharodon-carcharias', 'cardinalis-cardinalis', 'cathartes-aura', 'centrochelys-sulcata', 'centruroides-vittatus', 'ceratitis-capitata', 'ceratotherium-simum', 'chelonia-mydas', 'chrysemys-picta', 'circus-hudsonius', 'codium-fragile', 'coelacanthiformes', 'colaptes-auratus', 'connochaetes-gnou', 'correlophus-ciliatus', 'crocodylus-niloticus', 'crotalus-atrox', 'crotophaga-sulcirostris', 'cryptoprocta-ferox', 'cyanocitta-cristata', 'danaus-plexippus', 'dasypus-novemcinctus', 'delphinapterus-leucas', 'd

In [38]:
list(train.class_to_idx.keys())[10]

'aptenodytes-forsteri'

In [39]:
import timm
m = timm.create_model('mobilenetv3_large_100', pretrained=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mobilenetv3_large_100_ra-f55367f5.pth" to /home/hyojaejung/.cache/torch/hub/checkpoints/mobilenetv3_large_100_ra-f55367f5.pth


In [40]:
m

MobileNetV3(
  (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): Hardswish()
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (se): Identity()
        (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0): InvertedResidual(
        (conv_pw): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True